In [77]:
import numpy as np
from astropy.io import fits
import matplotlib.pyplot as plt
from scipy import stats
import numpy as np
from astropy.io import fits
import glob
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
import os
import math


In [78]:
model_A_path = '/global/cfs/cdirs/desi/spectro/desi_spectro_calib/trunk/'
model_B_path = '/global/cfs/cdirs/desi/users/schlafly/tpcorr-desi-spectro-calib/2comp/'
files_model_A = glob.glob(model_A_path+'spec/*/tpcorr*')
files_model_A.sort()
files_model_B = glob.glob(model_B_path+'spec/*/tpcorr*')
files_model_B.sort()

# Set to store unique outlier fiber IDs
outlier_fiber_ids = set()

for file_old in files_model_A:
    print(f"Processing: {file_old}")
    file_new = file_old.replace(model_A_path, model_B_path)
    
    # Load data
    spatial = fits.getdata(file_old, 'SPATIAL')
    fiber_id = spatial['FIBER']
    new = fits.getdata(file_old, 'MEAN')
    old = fits.getdata(file_new, 'MEAN')
    
    # Calculate differences
    diff = new - old
    
    # Use robust statistics for outlier detection
    median_diff = np.median(diff)
    mad = np.median(np.abs(diff - median_diff))
    modified_z_score = 0.6745 * (diff - median_diff) / mad
    
    # Identify outliers (e.g., modified Z-score > 5.0)
    outliers = np.abs(modified_z_score) > 5.0
    
    # Add outlier fiber IDs to the set
    outlier_fiber_ids.update(fiber_id[outliers])

# Convert set to sorted list
outlier_fiber_list = sorted(list(outlier_fiber_ids))

# Print summary
print(f"\nTotal number of unique outlier fibers: {len(outlier_fiber_list)}")
print("\nList of outlier fiber IDs:")
print(outlier_fiber_list)

# Optional: Save outlier fiber IDs to a file
with open('tpcorr_outlier_fibers.txt', 'w') as f:
    for fiber_id in outlier_fiber_list:
        f.write(f"{fiber_id}\n")

print(f"\nOutlier fiber IDs have been saved to 'tpcorr_outlier_fibers.txt'")

Processing: /global/cfs/cdirs/desi/spectro/desi_spectro_calib/trunk/spec/sm1/tpcorrparam-sm1-b4.fits
Processing: /global/cfs/cdirs/desi/spectro/desi_spectro_calib/trunk/spec/sm1/tpcorrparam-sm1-r4.fits
Processing: /global/cfs/cdirs/desi/spectro/desi_spectro_calib/trunk/spec/sm1/tpcorrparam-sm1-z4.fits
Processing: /global/cfs/cdirs/desi/spectro/desi_spectro_calib/trunk/spec/sm10/tpcorrparam-sm10-b1.fits
Processing: /global/cfs/cdirs/desi/spectro/desi_spectro_calib/trunk/spec/sm10/tpcorrparam-sm10-r1.fits
Processing: /global/cfs/cdirs/desi/spectro/desi_spectro_calib/trunk/spec/sm10/tpcorrparam-sm10-z1.fits
Processing: /global/cfs/cdirs/desi/spectro/desi_spectro_calib/trunk/spec/sm2/tpcorrparam-sm2-b8.fits
Processing: /global/cfs/cdirs/desi/spectro/desi_spectro_calib/trunk/spec/sm2/tpcorrparam-sm2-r8.fits
Processing: /global/cfs/cdirs/desi/spectro/desi_spectro_calib/trunk/spec/sm2/tpcorrparam-sm2-z8.fits
Processing: /global/cfs/cdirs/desi/spectro/desi_spectro_calib/trunk/spec/sm3/tpcorrpa

In [84]:

def plot_tpcorr_differences(files_model_A, files_model_B, output_file='tpcorr_differences.png'):
    n_files = len(files_model_A)
    n_cols = min(3, n_files)  # Maximum 3 columns
    n_rows = math.ceil(n_files / n_cols)

    fig, axs = plt.subplots(n_rows, n_cols, figsize=(2*n_cols, 2*n_rows), squeeze=False)
    fig.suptitle('TPCORR Differences: New - Old', fontsize=16)

    outlier_fiber_ids = set()

    for idx, (file_old, file_new) in enumerate(zip(files_model_A, files_model_B)):
        row = idx // n_cols
        col = idx % n_cols

        try:
            # Load data
            spatial = fits.getdata(file_old, 'SPATIAL')
            fiber_id = spatial['FIBER']
            new = fits.getdata(file_old, 'MEAN')
            old = fits.getdata(file_new, 'MEAN')

            # Calculate differences
            diff = new - old

            # Use robust statistics for outlier detection
            median_diff = np.median(diff)
            mad = np.median(np.abs(diff - median_diff))
            modified_z_score = 0.6745 * (diff - median_diff) / mad

            # Identify outliers (e.g., modified Z-score > 5.0)
            outliers = np.abs(modified_z_score) > 5.0

            # Plotting
            axs[row, col].scatter(fiber_id, diff, alpha=0.5, s=10, label='All fibers')
            axs[row, col].scatter(fiber_id[outliers], diff[outliers], color='red', s=5, label='Outliers')
            axs[row, col].set_title(os.path.basename(file_old), fontsize=8)
            axs[row, col].tick_params(axis='both', which='major', labelsize=6)
            
            if col == 0:
                axs[row, col].set_ylabel('Difference (New - Old)', fontsize=8)
            if row == n_rows - 1:
                axs[row, col].set_xlabel('Fiber ID', fontsize=8)

            outlier_fiber_ids.update(fiber_id[outliers])

        except Exception as e:
            print(f"Error processing {file_old}: {str(e)}")
            axs[row, col].text(0.5, 0.5, f"Error processing\n{os.path.basename(file_old)}", 
                               ha='center', va='center', transform=axs[row, col].transAxes)

    # Remove empty subplots
    for idx in range(n_files, n_rows * n_cols):
        row = idx // n_cols
        col = idx % n_cols
        fig.delaxes(axs[row][col])

    plt.tight_layout()
    plt.savefig(output_file, dpi=200, bbox_inches='tight')
    plt.close()

    print(f"Plot saved as {output_file}")

    return sorted(list(outlier_fiber_ids))

outlier_fiber_list = plot_tpcorr_differences(files_model_A, files_model_B)

Plot saved as tpcorr_differences.png
